In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import requests, json

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting  import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

In [2]:
def time_split(df, train_size = .5, validate_size = .3):
    '''Splits time series data based on percentages and returns train, validate, test THE
    DATAFRAME MUST BE CHRONOLOGICALLY SORTED!'''
    t_size = int(len(df) * train_size)
    v_size = int(len(df) * validate_size)
    end = t_size + v_size
    return df[0:t_size], df[t_size:end], df[end:len(df)+1]

def plot_samples(target_var):
    '''
   plot each attribute 
   '''
    plt.figure(figsize = (12,4))
    sns.lineplot(data=train[target_var], label='train')
    sns.lineplot(data=validate[target_var], label='validate')
    sns.lineplot(data=test[target_var], label='test')
    plt.title(target_var.title())
    plt.legend()
    
def evaluate(target_var):
    '''
    the evaluate function will take in the actual values and the predicted values
    and compute the mean_squared_error and then take the sqrt returning a rounded rmse
    '''
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])),0)
    return rmse

def plot_and_eval(target_var):
    '''
    a function to evaluate forecasts by computing the rmse and plot train and validate along with predictions
    '''
    plot_samples(target_var)
    sns.lineplot(data=yhat_df[target_var], label='RMSE')
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, f'--RMSE: {rmse:.0f}')
    plt.show()

def append_eval_df(model_type, target_var):
    '''
    this function will take in the model type as a string, target variable
    as a string, and run the evaluate function to compute rmse, 
    and append to the dataframe a row with the model type, 
    target variable and rmse. 
    '''
    rmse = evaluate(target_var)
    d= {'model_type':[model_type], 'target_var':[target_var], 'rmse':[rmse]}
    d= pd.DataFrame(d)
    return eval_df.append(d, ignore_index= True)

def previous_plot_and_eval(target_var):
    '''
    a function to evaluate forecasts by computing the rmse and plot train and validate along with predictions
    '''
    rmse = evaluate(target_var)
    print(target_var, f'--RMSE: {rmse:.0f}')
    return rmse

def fetch_data_dict(df):
    ''' Fetches and formats a data_dict to put into project README.md
    returns two data dict.
    '''
    data_dict = {
        'Timestamp' : 'start tiem of time window (60s window), in Unix Time',
        'Open':'Open price at start time window',
        'High':'High price within the time window',
        'Low':'Low price within the time window',
        'Close':'Close price at the end of the time window',
        'Volume_(BTC)':'Volume of BTC transacted in this window',
        'Volume_(Currency)':'Volume of corresponding currency transacted in this window',
        'Weighted_Price' :'VWAP - Volume Weighted Average Price'
    }

    data_dict = pd.DataFrame([{'Feature': col, 
         'Datatype': f'{df[col].count()} non-null: {df[col].dtype}',
        'Definition' : data_dict[col]} for col in df.columns]).set_index('Feature').to_markdown()
    return (data_dict)

In [18]:
df = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
df.Timestamp = pd.to_datetime(df.Timestamp, unit='s')
print(fetch_data_dict(df))
df = df.set_index('Timestamp')


| Feature           | Datatype                         | Definition                                                 |
|:------------------|:---------------------------------|:-----------------------------------------------------------|
| Timestamp         | 4857377 non-null: datetime64[ns] | start tiem of time window (60s window), in Unix Time       |
| Open              | 3613769 non-null: float64        | Open price at start time window                            |
| High              | 3613769 non-null: float64        | High price within the time window                          |
| Low               | 3613769 non-null: float64        | Low price within the time window                           |
| Close             | 3613769 non-null: float64        | Close price at the end of the time window                  |
| Volume_(BTC)      | 3613769 non-null: float64        | Volume of BTC transacted in this window                    |
| Volume_(Currency) | 3613769 non-null: float64        |

| Timestamp           |   Open |   High |    Low |   Close |   Volume_(BTC) |   Volume_(Currency) |   Weighted_Price |
|:--------------------|-------:|-------:|-------:|--------:|---------------:|--------------------:|-----------------:|
| 2011-12-31 07:52:00 |   4.39 |   4.39 |   4.39 |    4.39 |       0.455581 |                   2 |             4.39 |
| 2011-12-31 07:53:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |
| 2011-12-31 07:54:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |
| 2011-12-31 07:55:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |
| 2011-12-31 07:56:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |


## Calculate other metrics from data
> - Calculate month name, day names and price vairation from minute to minute

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4857377 entries, 2011-12-31 07:52:00 to 2021-03-31 00:00:00
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Open               float64
 1   High               float64
 2   Low                float64
 3   Close              float64
 4   Volume_(BTC)       float64
 5   Volume_(Currency)  float64
 6   Weighted_Price     float64
dtypes: float64(7)
memory usage: 296.5 MB


In [36]:
df = df.ffill()
df['day_of_week'] = df.index.day_name()
df['month'] = df.index.month_name()
df['minute_price_diff'] = df.Close - df.Open
df['intraday_high_low'] = df.High - df.Low
df['day_num'] = df.index.day

# Calculated from open
df['percent_change'] = (df.minute_price_diff / df.Open) * 100
df.tail()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,day_of_week,month,minute_price_diff,intraday_high_low,day_num,percent_change
Timestamp,,,,,,,,,,,,,
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,Tuesday,March,-28.31,28.31,30,-0.048217
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,Tuesday,March,1.84,9.46,30,0.003135
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,Tuesday,March,30.41,30.41,30,0.051812
2021-03-30 23:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202,Tuesday,March,18.41,28.20,30,0.031340
2021-03-31 00:00:00,58767.75,58778.18,58755.97,58778.18,2.712831,159417.751000,58764.349363,Wednesday,March,10.43,22.21,31,0.017748
